In [2]:
from google.colab import auth,drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install pythainlp
!pip install deepcut
!pip install stop_words
!pip install spacy
# !pip install pytorch

     |████████████████████████████████| 11.0MB 366kB/s 
     |████████████████████████████████| 747kB 24.2MB/s 
     |████████████████████████████████| 2.0MB 4.2MB/s 
  Created wheel for stop-words: filename=stop_words-2018.7.23-cp37-none-any.whl size=32913 sha256=f1e20b095b2a36620220bc49ef0a30fcdc364acb092319bf217ae30039b22e3a
  Stored in directory: /root/.cache/pip/wheels/75/37/6a/2b295e03bd07290f0da95c3adb9a74ba95fbc333aa8b0c7c78
Successfully built stop-words


In [4]:
import pandas as pd 
import numpy as np
from pythainlp.tokenize import word_tokenize
from pythainlp.tokenize import sent_tokenize
from pythainlp.tag.named_entity import ThaiNameTagger
from pythainlp.tag import pos_tag_sents
import deepcut
from pythainlp.corpus.common import thai_stopwords
from pythainlp.tag import pos_tag
from pythainlp.tag.named_entity import ThaiNameTagger

In [5]:
# tknzt = lambda x : word_tokenize(x,engine="newmm")
def tknzt(wrd) :
  ttkn =  word_tokenize(wrd,engine="newmm")
  return " ".join([i for i in ttkn if i.strip() != ""])

In [19]:
stopwords = list(thai_stopwords())
# stopwords

In [7]:
df = pd.read_csv("/content/drive/MyDrive/DEEP_LEARNING_PROJECT/DATA/adj_type_data_tkn.csv")

In [8]:
df

,Unnamed: 0.1,title,body,summary,type,tags,ไลฟ์สไตล์,เศรษฐกิจ,ต่างประเทศ,ความมั่นคง,สิ่งแวดล้อม,ผู้หญิง,การเมือง,แรงงาน,สังคม,อาชญากรรม,บันเทิง,สิทธิมนุษยชน,การศึกษา,วิทยาศาสตร์เทคโนโลยี,คุณภาพชีวิต,วัฒนธรรม,กีฬา,sum_hot,sum_hot_ag,body_cln
0,2,อังคณา เผยเตรียมร้อง ปอท. หลังถูกโพสต์สร้างควา...,โดยล่าสุดถูกปั่นว่าเรียกร้องให้สังคมให้อภัยเป...,อังคณา กสม. โพสต์ระบุเตรียมเข้าร้องต่อ ปอท.เรื...,"สิทธิมนุษยชน,ไอซีที","ปอท.,สร้างความเกลียดชัง,อังคณา นีละไพจิตร,เปรี้ยว",0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,2,2,โดยล่าสุดถูกปั่นว่าเรียกร้องให้สังคมให้อภัยเป...
1,4,คอนเต คนแรกทำสถิติให้ งูใหญ่ สตาร์ทเข้าวินรวด ...,สำนักข่าวต่างประเทศรายงานวันที่ 26 ก.ย. ว่า หล...,อันโตนิโอ คอนเต สร้างปรากฏการณ์ให้กับทัพ งูใหญ...,"กีฬา,ฟุตบอลยุโรป","อินเตอร์ มิลาน,อันโตนิโอ คอนเต,กัลโช เซเรีย อา...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,1,สำนักข่าวต่างประเทศรายงานวันที่ ก ย ว่า หลังจา...
2,9,เลือกตั้ง ผญบ.ใหม่ แทนผู้ใหญ่บัติ ตร.ยังคุ้มคร...,ชาวบ้านหมู่ 1 ต.บ้านกลาง อ.อ่าวลึก จ.กระบี่ ออ...,ชาวบ้านหมู่ 1 ต.บ้านกลาง อ.อ่าวลึก จ.กระบี่ ออ...,"ข่าว,อาชญากรรม","ฆ่ายกครัว,ฆ่า8ศพ,ผู้ใหญ่บัติ,เลือกตั้งผู้ใหญ่บ...",0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,2,1,ชาวบ้านหมู่ ต บ้านกลาง อ อ่าวลึก จ กระบี่ ออกม...
3,14,สมศักดิ์ เจียม ดันปัญญาชนไทยคิดยาว-ทำใหญ่ ปฏิร...,ชี้ถ้าไม่ยกเลิกองค์ประกอบอย่างมาตรา 8 ไทยก็ไม...,สมศักดิ์ เจียมธีรสกุล อภิปรายในงาน แขวนเสรีภาพ...,"การเมือง,สังคม","กฎหมายหมิ่นพระบรมเดชานุภาพ,กฎหมายอาญามาตรา 112...",0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,2,2,ชี้ถ้าไม่ยกเลิกองค์ประกอบอย่างมาตรา ไทยก็ไม่ม...
4,16,อดีต ผอ.กรมการคลังสินค้า ตั้งข้อสังเกตุเหตุไฟไ...,นายนิพนธ์ วงษ์ตระหง่าน อดีตผู้อำนวยการองค์การค...,ปัญหาไฟไหม้โรงสีหลายแห่ง ในเวลาใกล้เคียงกันใน...,เศรษฐกิจ,"กรมการคลังสินค้า,จำนำข้าว,ทุจริต,โรงสี,ไฟไหม้",0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,นายนิพนธ์ วงษ์ตระหง่าน อดีตผู้อำนวยการองค์การค...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188817,358858,19 ปีที่รอคอย อัล ฮิลาล บุกดับ อุราวะ เรด 2-0 ...,วันที่ 24 พ.ย. 62 การแข่งขันฟุตบอลที่ใหญ่สุดขอ...,อัล ฮิลาล ทีมดังจากประเทศซาอุดีอาระเบีย ผงาดคว...,"กีฬา,ฟุตบอลยุโรป","ข่าวบอล,ข่าวฟุตบอล,อุราวะ เรด ไดมอนส์,อัล ฮิลา...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,1,วันที่ พ ย การแข่งขันฟุตบอลที่ใหญ่สุดของทวีปเอ...
188818,358860,อนาคตใหม่ เผยตัวแทนทูต-นักวิชาการเตรียมสังเกตก...,รวมทั้งจะมีกลุ่มนักวิชาการและลูกศิษย์เดินทางไ...,โฆษกพรรคอนาคตใหม่เผย มีตัวแทนทูตของประเทศต่างๆ...,"การเมือง,สิทธิมนุษยชน","พรรณิการ์ วานิช,พรรคอนาคตใหม่,หมิ่นศาล,พ.ร.บ.ค...",0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,2,2,รวมทั้งจะมีกลุ่มนักวิชาการและลูกศิษย์เดินทางไ...
188819,358862,ที่แรกในเอเชีย ไต้หวันเอาจริง คนกินเนื้อหมา-แม...,"คุณ ไช่ อิง เหวิน, ที่ผลักดันกฏหมายการห้ามบริโ...",เมื่อกลางเดือนเมษายน ที่ไต้หวัน มีข่าวดีสำหรับ...,"คนดังนั่งเขียน,ข่าว","คนดังนั่งเขียน,นัชญ์ ทูนหัวของบ่าว,กินหมา,สัตว...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,1,"คุณ ไช่ อิง เหวิน, ที่ผลักดันกฏหมายการห้ามบริโ..."
188820,358864,ผิดคาด ไวจ์นัลดุม เผยแท็กติกหงส์แดง เกมบุกดวลเ...,สำนักข่าวต่างประเทศรายงาน วันที่ 9 เม.ย. ว่า จ...,จอร์จินิโอ ไวจ์นัลดุม มิดฟิลด์ทีมลิเวอร์พูล ออ...,"กีฬา,ฟุตบอลยุโรป","จอร์จินิโอ ไวจ์นัลดุม,ลิเวอร์พูล,แมนเชสเตอร์ ซ...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,1,สำนักข่าวต่างประเทศรายงาน วันที่ เม ย ว่า จอร์...


In [9]:
import re
import string

def remove_repettition(text):
    token_list = list(text)
    if len(token_list) > 2:
        filter_list = [True, True]
        n = len(token_list)
        for i in range(2, n):
            if (token_list[i] == token_list[i-1]) and (token_list[i] == token_list[i-2]):
                filter_list.append(False)
            else:
                filter_list.append(True)

        output = ''.join(np.array(token_list)[filter_list])
    else:
        output = text
    return output

def cleansing(text):
    # \t, \n, \xa0 and other special characters. Replace by blank string
    text = re.sub('[\t\n\xa0\"\'!?\/\(\)%\:\=\-\+\*\_ๆ]', '', text)
    
    # Numbers. Replace by space
    text = re.sub('[0-9]', ' ', text)
    
    # Dot. Replace by space
    text = re.sub('[\.]', ' ', text)
    
    # One or more consecutive space. Replace by single space
    text = re.sub('\s+',' ',text)
    
    # Remove 2 or more repettition
    text = remove_repettition(text)
    
    return text

In [ ]:
# df['body_cln'] = list(map(cleansing, df['body']))

In [13]:
# df = df.drop(["Unnamed: 0"],axis=1)
from pythainlp.corpus.common import thai_stopwords

In [10]:
df_test_01 = df.drop(["summary","tags"],axis=1)

In [11]:
df_test_01 = df_test_01.rename({"Unnamed: 0.1":"index"},axis=1)

In [12]:
df_test_01.index = df_test_01["index"]
df_test_01 = df_test_01.drop(["index"],axis=1)

In [20]:
df_test_01['Tkn_Text'] = list(map(tknzt, df_test_01['body_cln']))

In [22]:
df_test_01

,title,body,type,ไลฟ์สไตล์,เศรษฐกิจ,ต่างประเทศ,ความมั่นคง,สิ่งแวดล้อม,ผู้หญิง,การเมือง,แรงงาน,สังคม,อาชญากรรม,บันเทิง,สิทธิมนุษยชน,การศึกษา,วิทยาศาสตร์เทคโนโลยี,คุณภาพชีวิต,วัฒนธรรม,กีฬา,sum_hot,sum_hot_ag,body_cln,Tkn_Text
index,,,,,,,,,,,,,,,,,,,,,,,,
2,อังคณา เผยเตรียมร้อง ปอท. หลังถูกโพสต์สร้างควา...,โดยล่าสุดถูกปั่นว่าเรียกร้องให้สังคมให้อภัยเป...,"สิทธิมนุษยชน,ไอซีที",0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,2,2,โดยล่าสุดถูกปั่นว่าเรียกร้องให้สังคมให้อภัยเป...,โดย ล่าสุด ถูก ปั่น ว่า เรียกร้อง ให้ สังคม ให...
4,คอนเต คนแรกทำสถิติให้ งูใหญ่ สตาร์ทเข้าวินรวด ...,สำนักข่าวต่างประเทศรายงานวันที่ 26 ก.ย. ว่า หล...,"กีฬา,ฟุตบอลยุโรป",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,1,สำนักข่าวต่างประเทศรายงานวันที่ ก ย ว่า หลังจา...,สำนัก ข่าวต่างประเทศ รายงาน วันที่ ก ย ว่า หลั...
9,เลือกตั้ง ผญบ.ใหม่ แทนผู้ใหญ่บัติ ตร.ยังคุ้มคร...,ชาวบ้านหมู่ 1 ต.บ้านกลาง อ.อ่าวลึก จ.กระบี่ ออ...,"ข่าว,อาชญากรรม",0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,2,1,ชาวบ้านหมู่ ต บ้านกลาง อ อ่าวลึก จ กระบี่ ออกม...,ชาวบ้าน หมู่ ต บ้าน กลาง อ อ่าว ลึก จ กระบี่ อ...
14,สมศักดิ์ เจียม ดันปัญญาชนไทยคิดยาว-ทำใหญ่ ปฏิร...,ชี้ถ้าไม่ยกเลิกองค์ประกอบอย่างมาตรา 8 ไทยก็ไม...,"การเมือง,สังคม",0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,2,2,ชี้ถ้าไม่ยกเลิกองค์ประกอบอย่างมาตรา ไทยก็ไม่ม...,ชี้ ถ้า ไม่ ยกเลิก องค์ประกอบ อย่าง มาตรา ไทย ...
16,อดีต ผอ.กรมการคลังสินค้า ตั้งข้อสังเกตุเหตุไฟไ...,นายนิพนธ์ วงษ์ตระหง่าน อดีตผู้อำนวยการองค์การค...,เศรษฐกิจ,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,นายนิพนธ์ วงษ์ตระหง่าน อดีตผู้อำนวยการองค์การค...,นาย นิพนธ์ วงษ์ ตระหง่าน อดีต ผู้อำนวยการ องค์...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
358858,19 ปีที่รอคอย อัล ฮิลาล บุกดับ อุราวะ เรด 2-0 ...,วันที่ 24 พ.ย. 62 การแข่งขันฟุตบอลที่ใหญ่สุดขอ...,"กีฬา,ฟุตบอลยุโรป",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,1,วันที่ พ ย การแข่งขันฟุตบอลที่ใหญ่สุดของทวีปเอ...,วันที่ พ ย การแข่งขัน ฟุตบอล ที่ ใหญ่ สุด ของ ...
358860,อนาคตใหม่ เผยตัวแทนทูต-นักวิชาการเตรียมสังเกตก...,รวมทั้งจะมีกลุ่มนักวิชาการและลูกศิษย์เดินทางไ...,"การเมือง,สิทธิมนุษยชน",0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,2,2,รวมทั้งจะมีกลุ่มนักวิชาการและลูกศิษย์เดินทางไ...,รวมทั้ง จะ มี กลุ่ม นักวิชาการ และ ลูกศิษย์ เด...
358862,ที่แรกในเอเชีย ไต้หวันเอาจริง คนกินเนื้อหมา-แม...,"คุณ ไช่ อิง เหวิน, ที่ผลักดันกฏหมายการห้ามบริโ...","คนดังนั่งเขียน,ข่าว",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,1,"คุณ ไช่ อิง เหวิน, ที่ผลักดันกฏหมายการห้ามบริโ...","คุณ ไช่ อิง เห วิน , ที่ ผลักดัน กฏ หมาย การ ห..."


In [23]:
df_test_01.to_csv("/content/drive/MyDrive/DEEP_LEARNING_PROJECT/DATA/adj_type_data_tkn.csv",index=False)